# Voice recognition system to detect negative words spoken with a beep signal 

This algorithm receives input via microphone and processes it using different speech recognition services (e.g. Google and Sphinx). If any item of the negative words list is detected, it sends a beep signal as alert. 

First, the Speech Recognition package needs to be installed: 

In [2]:
!pip install --upgrade speechrecognition

Requirement already up-to-date: speechrecognition in /Users/Lucky1/anaconda/lib/python2.7/site-packages (3.8.1)


Then, Pyaudio is installed with the conda package manager: 

In [17]:
!conda install -y pyaudio 

Fetching package metadata ...........
Solving package specifications: .

Package plan for installation in environment /Users/Lucky1/anaconda:

The following NEW packages will be INSTALLED:

    portaudio: 19.6.0-h41429eb_1    

The following packages will be UPDATED:

    conda:     4.3.30-py27h407ed3a_0 --> 4.5.12-py27_0        
    conda-env: 2.6.0-0               --> 2.6.0-1              
    pyaudio:   0.2.7-py27_0          --> 0.2.11-py27h1de35cc_1
    pycosat:   0.6.1-py27_1          --> 0.6.3-py27h1de35cc_0 

conda-env-2.6. 100% |################################| Time: 0:00:00   1.81 MB/s
portaudio-19.6 100% |################################| Time: 0:00:00   4.45 MB/s
pyaudio-0.2.11 100% |################################| Time: 0:00:00  11.93 MB/s
pycosat-0.6.3- 100% |################################| Time: 0:00:00   4.86 MB/s
conda-4.5.12-p 100% |################################| Time: 0:00:00   3.30 MB/s


Implementation: 

In [1]:
# Code extended from: https://github.com/Uberi/speech_recognition/blob/master/examples/background_listening.py

import os
import time
import pyaudio
import speech_recognition as sr

# List of negative words
negative_words = ["if", "can't", "try"]

# Define stream for beep  
stream=pyaudio.PyAudio().open(format=pyaudio.paInt8,channels=1, rate=16000, output=True)

# Receives input per Mic and processes it using speech recognition services as background thread
def callback(recognizer, audio):
    
    # Using Sphinx Speech Recognition     
    #try:
    #    print(r.recognize_sphinx(audio))
    #except sr.UnknownValueError:
    #    print("Sphinx could not understand audio")
    #except sr.RequestError as e:
    #    print("Sphinx error; {0}".format(e))
    
    # Using Google Speech Recognition
    try:
        data = recognizer.recognize_google(audio)
        print(data)
        # Check for negative words
        for word in negative_words:
            if word in data: 
                print('Negative word detected: {}'.format(word))
                # Send the beep signal - system beep = os.system("\a")
                for n in range(200): 
                    stream.write("\x00\x30\x5a\x76\x7f\x76\x5a\x30\x00\xd0\xa6\x8a\x80\x8a\xa6\xd0")
            
    except sr.UnknownValueError:
        #print("Please say it again.")
        pass
    except sr.RequestError as e:
        print("Could not request results from Google Speech Recognition service, {0}".format(e))

r = sr.Recognizer()
m = sr.Microphone()
with m as source:
    print('Listening now..')
    # We only need to calibrate once, before we start listening
    r.adjust_for_ambient_noise(source) 

# Start listening in the background (note that we don't have to do this inside a 'with' statement)
stop_listening = r.listen_in_background(m, callback)

# Time in seconds listening in background
time.sleep(20)  

# Calling this function requests that the background function stops listening
stop_listening(wait_for_stop=False)

# Closing stream channel 
stream.close()
pyaudio.PyAudio().terminate()

print('Finished capturing voice.')

Listening now..
Santorini is totally awesome
yes she should try to go there
Negative word detected: try
Finished capturing voice.
